# Fetch Data Analyst Take Home Assessment: Canyen Heimuli

This notebook is my (Canyen Heimuli's) submission for the **take home assessment for the Fetch Senior Data Analyst role**. All coding and analysis work is completely my own.

In this notebook, I 1) explore the data to familiarize myself with tables, entity relationships, and any data quality issues; 2) extract some key insights from the data using Python and SQL; and 3) present some key takeaways about this data with a focus on actionable insights for an hypothetical manager or key stakeholder.

In [1]:
## Modules
import pandas as pd
import numpy as np
import os
import sqlite3
import datetime as dt

In [2]:
## Data
users = pd.read_csv('data/USER_TAKEHOME.csv')
products = pd.read_csv('data/PRODUCTS_TAKEHOME.csv')
transactions = pd.read_csv('data/TRANSACTION_TAKEHOME.csv')

## Exercise 1: Exploring the Data

### Question 1: Data Quality Issues

### Users Table

In [3]:
# Data Shape and Head
print(f"Table \"Users\" has shape {users.shape}")
users.head(10)

Table "Users" has shape (100000, 6)


,ID,CREATED_DATE,BIRTH_DATE,STATE,LANGUAGE,GENDER
0,5ef3b4f17053ab141787697d,2020-06-24 20:17:54.000 Z,2000-08-11 00:00:00.000 Z,CA,es-419,female
1,5ff220d383fcfc12622b96bc,2021-01-03 19:53:55.000 Z,2001-09-24 04:00:00.000 Z,PA,en,female
2,6477950aa55bb77a0e27ee10,2023-05-31 18:42:18.000 Z,1994-10-28 00:00:00.000 Z,FL,es-419,female
3,658a306e99b40f103b63ccf8,2023-12-26 01:46:22.000 Z,NaN,NC,en,NaN
4,653cf5d6a225ea102b7ecdc2,2023-10-28 11:51:50.000 Z,1972-03-19 00:00:00.000 Z,PA,en,female
5,5fe2b6f3ad416a1265c4ab68,2020-12-23 03:18:11.000 Z,1999-10-27 04:00:00.000 Z,NY,en,female
6,651210546816bb4d035b1ead,2023-09-25 22:57:24.000 Z,1983-09-25 22:57:25.000 Z,FL,es-419,male
7,642831ea3d4434e63c1936fd,2023-04-01 13:30:18.000 Z,1970-02-16 05:00:00.000 Z,IN,en,female
8,63a4c9a1b5f32149b9d82f9e,2022-12-22 21:18:25.000 Z,1982-12-22 05:00:00.000 Z,NC,en,female
9,63654b21d02459d8a57a2e2c,2022-11-04 17:25:53.000 Z,1992-05-03 04:00:00.000 Z,NY,en,non_binary


In the first 10 rows of the data there is a user record with a missing birthday. A substantial amount of missing birthdays will complicate analysis by age and there is no suitable imputation strategy for a missing birthday in this context.

I'll also check the distribution of missing/NA values in all fields:

In [4]:
# Missing value summary
users.isna().sum()

ID                  0
CREATED_DATE        0
BIRTH_DATE       3675
STATE            4812
LANGUAGE        30508
GENDER           5892
dtype: int64

While the amount of missing values is not insignificant, it alone doesn't suggest serious data quality issues to me. The "ID" field has no missing values; this is good, and it makes sense since this field is likely this table's primary key (in the context of the "data model" of these tables and information). I infer that it is the primary key both since it is referred to as an "ID" which keys typically are, and because it is a field that has a relationship with another table so it would likely need to be a key to prevent information loss on joins. The total number of missing "Birth Date" values is high but not significantly high. Still about 96% of users have a birthday, and about 95% still have "State" and "Gender" data.

Before moving on, I'll quickly do an informal check to verify that the "ID" field is a valid key (this is not something I would do if I were analyzing this data as already stored in a DBMS, but I wanted to check to ensure there would be no key/information loss problems when I load these tables into a database):

In [5]:
# Check that "ID" has all unique values
id_unique_vals_num = len(set(users['ID']))

if id_unique_vals_num == users.shape[0]:
    print('Field \"ID\" has all unique values')
else:
    print('Field \"ID\" has duplicate values')
    print(f"Number of unique values: {id_unique_vals_num}")
    print(f"Number of rows in Products table: {users.shape[0]}")

Field "ID" has all unique values


The amount of missing values for the "Language" field is potentially a cause for concern if only for its high incidence. In a table with 100,000 records, only about 70% have language information. Whether or not this is acceptable has to do with marketing as well as data pipelines. It might be assumed that those users can speak and read English, but if they do not, and increasing that sub-group's revenue or traffic is only a matter of updating their language information, it could be *well worth the investment to improve data collection and recording so that more accurate language data is collected for these users*. One potential reason for the missing data of all kinds could be that the user information is *derived from personal identification cards/documents* which may or may not be state-issued, may or may not have gender information, and may or may not be written in a recognizeable language.

With users being located across all states (as shown in the table `head()` call above), I'll investigate the distribution of languages in the table for those with language data:

In [6]:
# Distribution of languages
users.value_counts('LANGUAGE')

LANGUAGE
en        63403
es-419     6089
dtype: int64

According to the [U.S. Census](https://www.census.gov/quickfacts/fact/table/US/PST045224), around 75% of Americans identify as "white", with around 60% identifying as "white alone" (and not Hispanic/Latino). So while the distribution of language data roughly tracks with the Census (making a simplifying assumption that all who have their language set to "English" are white), it is highly unlikely that all respondents who have missing language data don't read or speak any English. That said, it still could be worthwhile to either 1) look into these user records and investigate what their language information ought to be, or 2) improve data collection processes to reduce the proportion of missing language data.

With regard to analysis in this assessment, I will make the simplifying assumption that **all missing values are an intentional representation of reality** and **keep them in the data but filter them out of the data before analysis dealing specifically with age groups**. 

### Products Table

In [7]:
# Data Shape and Head

print(f"Table \"Products\" has shape {products.shape}")
products.head(10)

Table "Products" has shape (845552, 7)


,CATEGORY_1,CATEGORY_2,CATEGORY_3,CATEGORY_4,MANUFACTURER,BRAND,BARCODE
0,Health & Wellness,Sexual Health,Conductivity Gels & Lotions,NaN,NaN,NaN,7.964944e+11
1,Snacks,Puffed Snacks,Cheese Curls & Puffs,NaN,NaN,NaN,2.327801e+10
2,Health & Wellness,Hair Care,Hair Care Accessories,NaN,PLACEHOLDER MANUFACTURER,ELECSOP,4.618178e+11
3,Health & Wellness,Oral Care,Toothpaste,NaN,COLGATE-PALMOLIVE,COLGATE,3.500047e+10
4,Health & Wellness,Medicines & Treatments,Essential Oils,NaN,MAPLE HOLISTICS AND HONEYDEW PRODUCTS INTERCHA...,MAPLE HOLISTICS,8.068109e+11
5,Health & Wellness,Hair Care,Hair Care Accessories,NaN,PLACEHOLDER MANUFACTURER,BEAUHAIR,6.626585e+11
6,Health & Wellness,Medicines & Treatments,Vitamins & Herbal Supplements,NaN,HALEON,EMERGEN-C,6.177376e+11
7,Health & Wellness,Deodorant & Antiperspirant,Men's Deodorant & Antiperspirant,NaN,NaN,NaN,7.501839e+12
8,Snacks,Snack Bars,Granola Bars,NaN,HYVEE INC,HY-VEE,7.545013e+10
9,Health & Wellness,NaN,NaN,NaN,CHURCH & DWIGHT,REPHRESH,NaN


This table also has several missing datapoints across multiple fields. Here is a summary of the number of NA values:

In [8]:
# Missing value summary
products.isna().sum()

CATEGORY_1         111
CATEGORY_2        1424
CATEGORY_3       60566
CATEGORY_4      778093
MANUFACTURER    226474
BRAND           226472
BARCODE           4025
dtype: int64

Nearly all products have at least one category and it's reasonable that most products would have both multiple and only 2 at most. The proportion of records missing the 3rd and 4th categories is not a concern to me. About 1 in 4 records are missing both the "Manufacturer" and "Brand" fields. My guess is that this data is missing for so-called "generic" products that are manufactured by retailers themselves and have no brand, or in any case that the missing-ness of this data is intentional and not a data collection issue. As with missing "Birth Dates", no common imputation strategy makes sense so I won't use any.

Another cause for concern is that there is no field that clearly communicates the product's *name* or *what the product is* -- at least not in one field or a combination of fields that is eminently obvious. For some records (looking at the selected rows above), the nature of the product is mostly communicated across all category fields put together, but this is not always the case. Additional information can be gleaned from the "Manufacturer" and "Brand" fields where they are present, but these likewise do not definitively communicate the product's name or product type. This will make reporting to company stakeholders very cumbersome as while analysis will be aggregated by the "Barcode" field, showing the barcodes alone does not fully communicate the most actionable business insights. Showing all product categories, manufacturing, and brand information can suffice, and will have to in this assessment, but is intractable for quickly and succinctly reporting key findings.

The biggest cause for concern, however, is that the **"Barcode" field is missing for a few thousand records**. Since this is likely the primary key of this table, these records can't be treated as meaningful in any analysis, and they technically should not exist anyway in an internally consistent database mangement system (DBMS). With regard to analysis in this assessment, I will make the simplifying assumption that **these records are faulty and will delete them before conducting analysis**. 

In [9]:
# Remove NA values for "Barcode" field
products_cleaned = products \
    .dropna(subset = ['BARCODE']) \
    .reset_index(drop = True)

Last, I'll do an informal check that the "Barcode" field is a valid key:

In [10]:
# Check that "Barcode" has all unique values
barcode_unique_vals_num = len(set(products_cleaned['BARCODE']))

if barcode_unique_vals_num == products_cleaned.shape[0]:
    print('Field \"Barcode\" has all unique values')
else:
    print('Field \"Barcode\" has duplicate values')
    print(f"Number of unique values: {barcode_unique_vals_num}")
    print(f"Number of rows in Products table: {products_cleaned.shape[0]}")

Field "Barcode" has duplicate values
Number of unique values: 841342
Number of rows in Products table: 841527


After removing NA values for the "Barcode" field, there are still duplicate values for this field. I'll investigate a few of them:

In [11]:
# Rows with duplicate "Barcode" values
products_cleaned[products_cleaned.duplicated(subset = ['BARCODE'], keep = False)] \
    .sort_values(['BARCODE']) \
    .head(20)

,CATEGORY_1,CATEGORY_2,CATEGORY_3,CATEGORY_4,MANUFACTURER,BRAND,BARCODE
348328,Snacks,Candy,Confection Candy,NaN,MARS WRIGLEY,STARBURST,400510.0
99091,Snacks,Candy,Confection Candy,NaN,MARS WRIGLEY,STARBURST,400510.0
138459,Snacks,Candy,Chocolate Candy,NaN,PLACEHOLDER MANUFACTURER,BRAND NOT KNOWN,404310.0
837226,Snacks,Candy,Chocolate Candy,NaN,MARS WRIGLEY,M&M'S,404310.0
273036,Snacks,Crackers,Graham Crackers,NaN,TRADER JOE'S,TRADER JOE'S,438711.0
681394,Snacks,Crackers,Graham Crackers,NaN,TRADER JOE'S,TRADER JOE'S,438711.0
484677,Snacks,Fruit & Vegetable Snacks,Dried Vegetables,NaN,TRADER JOE'S,TRADER JOE'S,563178.0
740071,Snacks,Fruit & Vegetable Snacks,Dried Vegetables,NaN,TRADER JOE'S,TRADER JOE'S,563178.0
268084,Snacks,Nuts & Seeds,Cashews,NaN,TRADER JOE'S,TRADER JOE'S,603898.0
752505,Snacks,Nuts & Seeds,Cashews,NaN,TRADER JOE'S,TRADER JOE'S,603898.0


This is a serious problem. Not only are there duplicate values for this field, but from this limited view of the data, the duplicates are different in ways that do not denote one single likely cause, nor a single solution. Many records are duplicates with regard to every field in the table, and it can be assumed the duplicate rows for these records were eronneously recorded twice. These duplicates will be easy enough to deal with. But other duplicate records say both a company in the "Brand" field and the text "Brand not known". Still other duplicates contain two different brands which are not the same with no way to tell between them which is correct.

While this is not an ideal solution, I will implement a working approach where I **remove duplicate rows that are the same with respect to all fields including "Manufacturer" and "Brand"**, and also **remove rows that have conflicting values for other fields and set any and all conflicting fields to "NA"**. This is an approach that I believe will preserve the maximum amount of information as it will not result in having to scrub any rows with unique "Barcode" values. In the real world, this is a kind of problem that I would address in the data pipeline/ingestion level.

In [12]:
# Handle Duplicates Function
def handle_duplicate_rows(df, id_column):
    # Scope for handling one group
    def process_group(group):
        if len(group) == 1:
            return group

        first_row = group.iloc[0].drop(id_column)
        diff_cols = []

        for col in group.columns:
            if col != id_column:
                if not all(group[col].eq(first_row[col])):
                    diff_cols.append(col)

        if not diff_cols:
            return group.drop_duplicates()
        else:
            for col in diff_cols:
                group[col] = np.nan
            return group.drop_duplicates(subset = [id_column])

    return df.groupby(id_column, group_keys = False).apply(process_group)

In [13]:
# Dealing with Product duplicates
products_duplicates = products_cleaned[products_cleaned.duplicated(subset = ['BARCODE'], keep = False)]
products_duplicates_fixed = handle_duplicate_rows(products_duplicates, 'BARCODE')

products_cleaned = pd.concat([products_cleaned.drop_duplicates(subset = ['BARCODE'], keep = False), products_duplicates_fixed], ignore_index = True)

In [14]:
# Check that "Barcode" has all unique values
barcode_unique_vals_num = len(set(products_cleaned['BARCODE']))

if barcode_unique_vals_num == products_cleaned.shape[0]:
    print('Field \"Barcode\" has all unique values')
else:
    print('Field \"Barcode\" has duplicate values')
    print(f"Number of unique values: {barcode_unique_vals_num}")
    print(f"Number of rows in Products table: {products_cleaned.shape[0]}")

Field "Barcode" has all unique values


### Transactions Table

In [15]:
# Data Shape and Head
print(f"Table \"Transactions\" has shape {transactions.shape}")
transactions.head(10)

Table "Transactions" has shape (50000, 8)


,RECEIPT_ID,PURCHASE_DATE,SCAN_DATE,STORE_NAME,USER_ID,BARCODE,FINAL_QUANTITY,FINAL_SALE
0,0000d256-4041-4a3e-adc4-5623fb6e0c99,2024-08-21,2024-08-21 14:19:06.539 Z,WALMART,63b73a7f3d310dceeabd4758,1.530001e+10,1.00,
1,0001455d-7a92-4a7b-a1d2-c747af1c8fd3,2024-07-20,2024-07-20 09:50:24.206 Z,ALDI,62c08877baa38d1a1f6c211a,NaN,zero,1.49
2,00017e0a-7851-42fb-bfab-0baa96e23586,2024-08-18,2024-08-19 15:38:56.813 Z,WALMART,60842f207ac8b7729e472020,7.874223e+10,1.00,
3,000239aa-3478-453d-801e-66a82e39c8af,2024-06-18,2024-06-19 11:03:37.468 Z,FOOD LION,63fcd7cea4f8442c3386b589,7.833997e+11,zero,3.49
4,00026b4c-dfe8-49dd-b026-4c2f0fd5c6a1,2024-07-04,2024-07-05 15:56:43.549 Z,RANDALLS,6193231ae9b3d75037b0f928,4.790050e+10,1.00,
5,0002d8cd-1701-4cdd-a524-b70402e2dbc0,2024-06-24,2024-06-24 19:44:54.247 Z,WALMART,5dcc6c510040a012b8e76924,6.811314e+11,zero,1.46
6,000550b2-1480-4c07-950f-ff601f242152,2024-07-06,2024-07-06 19:27:48.586 Z,WALMART,5f850bc9cf9431165f3ac175,4.920091e+10,1.00,
7,00096c49-8b04-42f9-88ce-941c5e06c4a7,2024-08-19,2024-08-21 17:35:21.902 Z,TARGET,6144f4f1f3ef696919f54b5c,7.830007e+10,zero,3.59
8,000e1d35-15e5-46c6-b6b3-33653ed3d27e,2024-08-13,2024-08-13 18:21:07.931 Z,WALMART,61a6d926f998e47aad33db66,5.200001e+10,1.00,
9,0010d87d-1ad2-4e5e-9a25-cec736919d15,2024-08-04,2024-08-04 18:01:47.787 Z,ALDI,66686fc2e04f743a096ea808,NaN,zero,2.29


This table shows transactions and has two fields, "User_ID" and "Barcode", that are foreign keys in the "Users" and "Products" tables according to the ER diagram. Its key is likely the "Receipt_ID" field though I will investigate this later.

Here is the distribution of missing values:

In [16]:
# Missing value summary
transactions.isna().sum()

RECEIPT_ID           0
PURCHASE_DATE        0
SCAN_DATE            0
STORE_NAME           0
USER_ID              0
BARCODE           5762
FINAL_QUANTITY       0
FINAL_SALE           0
dtype: int64

The "Barcode" field is missing for a few thousand records. It is unclear how this kind of recording error could have come about since each record does have a scan date, receipt ID, store name, user, and price. In any case, these kinds of records couldn't be used in analysis dis-aggregated by user or some user-specific demographic, or dis-aggregated by product. So these records will be **removed before conducting analysis**.

I'll also verify that the "Receipt ID" field is a valid key:

In [17]:
# Check that "Receipt ID" has all unique values
receipts_unique_vals_num = len(set(transactions['RECEIPT_ID']))

if receipts_unique_vals_num == transactions.shape[0]:
    print('Field \"Receipt ID\" has all unique values')
else:
    print('Field \"Receipt ID\" has duplicate values')
    print(f"Number of unique values: {receipts_unique_vals_num}")
    print(f"Number of rows in Transactions table: {transactions.shape[0]}")

Field "Receipt ID" has duplicate values
Number of unique values: 24440
Number of rows in Transactions table: 50000


Not only does this table contain duplicate records, but it contains tens of thousands of them. I visually investigate a handful of them with the code below:

In [18]:
# Rows with duplicate "Receipt ID" values
transactions[transactions.duplicated(subset = ['RECEIPT_ID'], keep = False)] \
    .sort_values(['RECEIPT_ID']) \
    .head(20)

,RECEIPT_ID,PURCHASE_DATE,SCAN_DATE,STORE_NAME,USER_ID,BARCODE,FINAL_QUANTITY,FINAL_SALE
0,0000d256-4041-4a3e-adc4-5623fb6e0c99,2024-08-21,2024-08-21 14:19:06.539 Z,WALMART,63b73a7f3d310dceeabd4758,1.530001e+10,1.00,
41567,0000d256-4041-4a3e-adc4-5623fb6e0c99,2024-08-21,2024-08-21 14:19:06.539 Z,WALMART,63b73a7f3d310dceeabd4758,1.530001e+10,1.00,1.54
1,0001455d-7a92-4a7b-a1d2-c747af1c8fd3,2024-07-20,2024-07-20 09:50:24.206 Z,ALDI,62c08877baa38d1a1f6c211a,NaN,zero,1.49
39291,0001455d-7a92-4a7b-a1d2-c747af1c8fd3,2024-07-20,2024-07-20 09:50:24.206 Z,ALDI,62c08877baa38d1a1f6c211a,NaN,1.00,1.49
2,00017e0a-7851-42fb-bfab-0baa96e23586,2024-08-18,2024-08-19 15:38:56.813 Z,WALMART,60842f207ac8b7729e472020,7.874223e+10,1.00,
25928,00017e0a-7851-42fb-bfab-0baa96e23586,2024-08-18,2024-08-19 15:38:56.813 Z,WALMART,60842f207ac8b7729e472020,7.874223e+10,1.00,2.54
3,000239aa-3478-453d-801e-66a82e39c8af,2024-06-18,2024-06-19 11:03:37.468 Z,FOOD LION,63fcd7cea4f8442c3386b589,7.833997e+11,zero,3.49
41475,000239aa-3478-453d-801e-66a82e39c8af,2024-06-18,2024-06-19 11:03:37.468 Z,FOOD LION,63fcd7cea4f8442c3386b589,7.833997e+11,1.00,3.49
43233,00026b4c-dfe8-49dd-b026-4c2f0fd5c6a1,2024-07-04,2024-07-05 15:56:43.549 Z,RANDALLS,6193231ae9b3d75037b0f928,4.790050e+10,1.00,5.29
4,00026b4c-dfe8-49dd-b026-4c2f0fd5c6a1,2024-07-04,2024-07-05 15:56:43.549 Z,RANDALLS,6193231ae9b3d75037b0f928,4.790050e+10,1.00,


From this limited view, several pairs of rows are identical with regard to multiple fields, including "Receipt ID", "Purchase Date", "Scan Date", "Store Name", "User ID", and "Barcode". Even missing values, where present, are identical within pairs of rows. The only fields where the pairs of rows differ are for the "Quantity" and "Sale" fields.

In my estimation, this data is *faulty* because of, or as evidenced by these semi-duplicate rows, and that the row duplication is not genuine or a true representation of reality. One possible way this data table could have been formed was by different systems recording the same receipt scanning multiple times.

Based on this limited view, I'll make a simplifying assumption that **no record should have a missing "Final Sale" value so I will start cleaning this data by removing null values of this field**. Likewise, an empty "Final Quantity" field is functionally meaningless in a record of transactions (there is no need to record a transaction where the quantity of items sold was "zero"), so I will **also remove rows where this field has a value of "zero"**.

In [19]:
# Clean Transactions data
transactions_cleaned = transactions.copy()

# Remove null sales values
transactions_cleaned['FINAL_SALE'] = pd.to_numeric(transactions_cleaned['FINAL_SALE'], errors = 'coerce')
transactions_cleaned = transactions_cleaned[~np.isnan(transactions_cleaned['FINAL_SALE'])]

# Remove "zero" quantity values
transactions_cleaned = transactions_cleaned[transactions_cleaned['FINAL_QUANTITY'] != 'zero']
transactions_cleaned['FINAL_QUANTITY'] = pd.to_numeric(transactions_cleaned['FINAL_QUANTITY'], errors = 'coerce')

I'll also remove the records with null "Barcode" values which I found earlier:

In [20]:
# Remove null Barcode values
transactions_cleaned = transactions_cleaned[~np.isnan(transactions_cleaned['BARCODE'])]

In [21]:
# Check that "Receipt ID" has all unique values
receipts_unique_vals_num = len(set(transactions_cleaned['RECEIPT_ID']))

if receipts_unique_vals_num == transactions_cleaned.shape[0]:
    print('Field \"Receipt ID\" has all unique values')
else:
    print('Field \"Receipt ID\" has duplicate values')
    print(f"Number of unique values: {receipts_unique_vals_num}")
    print(f"Number of rows in Transactions table: {transactions_cleaned.shape[0]}")

Field "Receipt ID" has duplicate values
Number of unique values: 21639
Number of rows in Transactions table: 22119


After this, there are still about 500 records with duplicate "Receipt ID"s. I visually examine a few of them with the code below:

In [22]:
# Rows with duplicate "Receipt ID" values
transactions_cleaned[transactions_cleaned.duplicated(subset = ['RECEIPT_ID'], keep = False)] \
    .sort_values(['RECEIPT_ID']) \
    .head(20)

,RECEIPT_ID,PURCHASE_DATE,SCAN_DATE,STORE_NAME,USER_ID,BARCODE,FINAL_QUANTITY,FINAL_SALE
40498,007d3232-3990-497f-a081-549e9e7a478b,2024-06-25,2024-06-27 21:21:53.442 Z,DOLLAR TREE STORES INC,63a8dbf101cb7c888c6ad87d,7.920006e+10,1.0,1.25
45553,007d3232-3990-497f-a081-549e9e7a478b,2024-06-25,2024-06-27 21:21:53.442 Z,DOLLAR TREE STORES INC,63a8dbf101cb7c888c6ad87d,7.920006e+10,1.0,1.25
45071,019520fd-f28b-42ff-a29e-d6bf4f5c4b7c,2024-08-19,2024-08-19 11:19:59.681 Z,MEIJER,646bdaa67a342372c857b958,4.125009e+10,1.0,2.49
41005,019520fd-f28b-42ff-a29e-d6bf4f5c4b7c,2024-08-19,2024-08-19 11:19:59.681 Z,MEIJER,646bdaa67a342372c857b958,4.125022e+10,1.0,3.79
49759,01a70fe0-026f-4bea-9da4-7d13bbf21e9a,2024-09-02,2024-09-07 16:02:39.835 Z,WALMART,614e733372ba844aa8dc345e,4.178900e+10,1.0,0.52
49758,01a70fe0-026f-4bea-9da4-7d13bbf21e9a,2024-09-02,2024-09-07 16:02:39.835 Z,WALMART,614e733372ba844aa8dc345e,4.178900e+10,1.0,0.52
35488,01d0e64c-0b62-405f-a7bf-2dc04c36a53d,2024-08-23,2024-08-26 10:05:19.028 Z,WINCO FOODS,6079c0e9116de356d8ad6485,5.150004e+10,1.0,3.83
34963,01d0e64c-0b62-405f-a7bf-2dc04c36a53d,2024-08-23,2024-08-26 10:05:19.028 Z,WINCO FOODS,6079c0e9116de356d8ad6485,1.600021e+10,1.0,2.62
32463,0273cbd8-1620-46c9-8e99-6971e850a2fc,2024-09-08,2024-09-08 22:17:11.989 Z,WALMART,60e4f3ac34f82e1344669ee2,6.811311e+11,1.0,3.48
32462,0273cbd8-1620-46c9-8e99-6971e850a2fc,2024-09-08,2024-09-08 22:17:11.989 Z,WALMART,60e4f3ac34f82e1344669ee2,6.811311e+11,1.0,3.48


These duplicate records look similar in style to the duplicate records in the "Products" table. I will deal with them the same way:

In [23]:
# Resolve duplicates
transactions_cleaned_deduped = handle_duplicate_rows(transactions_cleaned, 'RECEIPT_ID')

In [24]:
# Check that "Receipt ID" has all unique values
receipts_unique_vals_num = len(set(transactions_cleaned_deduped['RECEIPT_ID']))

if receipts_unique_vals_num == transactions_cleaned_deduped.shape[0]:
    print('Field \"Receipt ID\" has all unique values')
else:
    print('Field \"Receipt ID\" has duplicate values')
    print(f"Number of unique values: {receipts_unique_vals_num}")
    print(f"Number of rows in Transactions table: {transactions_cleaned_deduped.shape[0]}")

Field "Receipt ID" has all unique values


The "Receipt ID" field for the Transactions table is now a valid key. The duplicate resolution process induced some extra "NA" values in the "Barcode" field for duplicate rows that had differing values for that field. They will also have to be deleted:

In [25]:
# Remove new null Barcode values
transactions_cleaned_deduped = transactions_cleaned_deduped[~np.isnan(transactions_cleaned_deduped['BARCODE'])]

To recap, each table had a handful of data quality issues that raised concerns for me about how this data was ingested and validated in its DBMS or equivalent data processing infrastructure. The Users table didn't have significant data quality issues with regard to its structural integrity (the "ID" field was already a valid key from the start), but the large proportion of records with missing "Language" data should be investigated. The Products table had a small number of duplicates with regard to its primary key "Barcode", and the Transactions table had a significant amount of duplicates with regard to its primary key "Receipt ID". Both of these are causes for concern and suggest that duplicate records are being entered into the system, and that the DBMS or equivalent data processing system is not configured to handle these cases. Another big issue not related to data integrity specifically but which is important is that the Products table has no field or collection of fields that definiitively communicate what the product is. This makes reporting on product-specific metrics intractable and may lead to some awkward meetings with non-technical audiences who are familiar with products only by their names. 

I have implemented some cosmetic measures as a workaround and to minimize errors with running SQL queries, but in a genuine setting, I would address these problems at the source, and address them or make sure they were addressed by data engineers as quickly as possible.

## Exercise 2: SQL Queries

Now that I've done some data cleaning, I load these tables into a database to analyze them using SQL with the code below:

*A Note on implementation: In this exercise, I opted to use SQLite for its ease of execution within a Python kernel environment without the need to host a database on a server or application, use containerization, or use a driver. While I use SQLite and its specific syntax in this exercise, I also have project experience in MySQL and MySQL Workbench as well as MS SQL Server.*

In [26]:
## Load Tables into DB for Querying
# Overwrite table names for ease
products = products_cleaned.copy()
transactions = transactions_cleaned_deduped.copy()

# Create DB; Define filepath
db_file = 'fetch.db'

# Connect to DB
conn = sqlite3.connect(db_file)
cursor = conn.cursor()

# Write Table to SQLite DB
tables = [users, products, transactions]
table_names = ['USERS', 'PRODUCTS', 'TRANSACTIONS']

for table, name in zip(tables, table_names):
    table.to_sql(name, conn, if_exists = 'replace', index = False)
    print(f"Successfully imported table '{name}'.")

# Close connection
conn.commit()
conn.close()

Successfully imported table 'USERS'.
Successfully imported table 'PRODUCTS'.
Successfully imported table 'TRANSACTIONS'.


In [27]:
## Convenience Function for Visualizing Queries
def query_result(query):
    conn = sqlite3.connect('fetch.db')
    result = pd.read_sql_query(query, conn)
    conn.close()
    return result

### Close-Ended

#### Question 1: Top 5 brands by Receipts Among Users 21 and over

In [28]:
# Top 5 brands by Receipts Among Users 21 and over
query = """
  SELECT P.BRAND, COUNT(*) AS TOTAL_RECEIPTS
    FROM TRANSACTIONS T
    LEFT JOIN PRODUCTS P ON T.BARCODE = P.BARCODE
    LEFT JOIN USERS U ON T.USER_ID = U.ID
    WHERE P.BRAND != "None" AND (2025 - STRFTIME('%Y', U.BIRTH_DATE)) >= 21
    GROUP BY P.BRAND
    ORDER by TOTAL_RECEIPTS DESC, BRAND
    LIMIT 5;
"""

query_result(query)

,BRAND,TOTAL_RECEIPTS
0,DOVE,3
1,NERDS CANDY,3
2,COCA-COLA,2
3,GREAT VALUE,2
4,HERSHEY'S,2


Among users 21 and over, the top brands by receipts scanned are the "Dove", "Nerds", "Coca-Cola", "Great Value", and "Hershey's" brands. Three of these brands are candy brands on the more affordable end, and another is the name brand of goods manufactured and sold by Walmart. That many of the top brands are both candy and cheaper brands is a valuable insight.

I can verify these results by extracting them using Python as well:

In [29]:
## CE Q1 in Python
# Products filtered
products_filtered = products[products['BRAND'].notna()]

# Users filtered
users_filtered = users.loc[:, ['BIRTH_DATE', 'ID']].copy()
users_filtered = users_filtered[users_filtered['BIRTH_DATE'].notna()]
users_filtered['BIRTH_DATE'] = pd.to_datetime(users_filtered['BIRTH_DATE'])
users_filtered['AGE'] = (dt.datetime.now().year - users_filtered['BIRTH_DATE'].dt.year)
users_filtered = users_filtered[users_filtered['AGE'] >= 21]

# Join to Transactions table
full_data = pd.merge(transactions, products_filtered, on = 'BARCODE', how = 'inner')
full_data = pd.merge(full_data, users_filtered, left_on = 'USER_ID', right_on = 'ID', how = 'inner')

# Summarize data
full_data.groupby('BRAND').size() \
    .reset_index(name = 'TOTAL_RECEIPTS') \
    .sort_values(['TOTAL_RECEIPTS', 'BRAND'], ascending = [False, True]) \
    .head(5)

,BRAND,TOTAL_RECEIPTS
13,DOVE,3
35,NERDS CANDY,3
6,COCA-COLA,2
21,GREAT VALUE,2
22,HERSHEY'S,2


I implemented an ordering on the "Brand" column for analysis in both languages to show that the results are the same.

These final results may not look intuitive in terms of the number of total receipts, but they are a genuine summary of the filtered transactions dataset with non-null product barcodes and users over 21 years of age. That dataset had a very small number of records leading to a likewise small number of total receipts among even the most popular brands. One second-order finding, then, could be that the company's user base of those aged 21 or over does not have a lot of shopping traffic.

*Note that these findings are also a product of the simplifying assumptions I made about the data in preparation for analysis. Though the query satisfies the assignment and business question, it may lead to the wrong result if I have made a mistake in data pre-processing.*

#### Question 2: Top 5 brands by Sales Among Mature Users (6 Months or Older)

In [30]:
# Top 5 brands by Sales Among Mature Users (6 Months-old Account or Older)
query = """
  SELECT P.BRAND, SUM(FINAL_SALE) AS TOTAL_SALES
    FROM TRANSACTIONS T
    LEFT JOIN PRODUCTS P ON T.BARCODE = P.BARCODE
    LEFT JOIN USERS U ON T.USER_ID = U.ID
    WHERE P.BRAND != "None" AND ((JULIANDAY("now") - JULIANDAY(CREATED_DATE)) / 30.4) > 6
    GROUP BY P.BRAND
    ORDER by TOTAL_SALES DESC
    LIMIT 5;
"""

query_result(query)

,BRAND,TOTAL_SALES
0,CVS,72.00
1,DOVE,30.91
2,TRIDENT,23.36
3,COORS LIGHT,17.48
4,TRESEMMÉ,14.58


Among account holders who have had their accounts for 6 months or longer, the top brands are "CVS", "Dove", "Trident", "Coors Light", and "Tresemme" in that order. One thematic throughline that these brands' products have in common is they are often "convenience store" products, or products you might find at a corner store or gas station. CVS stores often have convenience food stuffs including frozen foods and snacks, while Dove and Trident candies are often found in grocery stores near the checkout lanes.

### Open-Ended

#### Question 2: The Leading Brand in the Dips & Salsa Category

The term "Leading Brand" isn't a set term with one definition, but I will operationalize it by **amount of receipts scanned**. I make a simplifying assumption that this is the metric that has the most applicability to Fetch. 

Dips vary widely in price and quality, with those two things hopefully being positively correlated. Some name brand or low quality dips could be on the order of \\$2 of \\$3. In New England where I currently live, one of the most well-liked and high quality dips is Maryland Crab Dip, known for its unique taste and high potential for pairings with other foods. In this case, amount of sales or "market share" are, in my simplifying assumption, less important than the number of receipts that users scan on the Fetch app.

The "Dips & Salsa" category is a "Category 2" value and sub-category of the "Snacks" category contained in the "Category 1" field. The query below returns the result:

In [31]:
# Leading Brand(s) by Receipts in the "Dips and Salsa" Category
query = """
  SELECT P.BRAND, COUNT(*) AS TOTAL_RECEIPTS
    FROM TRANSACTIONS T
    LEFT JOIN PRODUCTS P ON T.BARCODE = P.BARCODE
    WHERE P.BRAND != "None" AND P.CATEGORY_2 = 'Dips & Salsa'
    GROUP BY P.BRAND
    ORDER BY TOTAL_RECEIPTS DESC, BRAND
    LIMIT 5;
"""

query_result(query)

,BRAND,TOTAL_RECEIPTS
0,TOSTITOS,34
1,PACE,22
2,DEAN'S DAIRY DIP,17
3,FRITOS,17
4,HELUVA GOOD!,14


So in this database, the leading brand is "Tostitos". This brand is followed somewhat closely by "Pace", "Dean's Dairy Dip", "Fritos", and "Heluva Good!", though Tostito's has a solid lead over Pace at least in this databse.

## Exercise 3: Communicating with Stakeholders

Given the state of this database and my findings from exploratory data analysis, here is an email or Slack message I would send to a product or business leader summarizing the results of my investigation:

___
*Dear Manager,*

*I have just completed some preliminary analysis of our database of users, products, and transactions. While I uncovered some important data quality issues in our database that will need to addressed, I find that some top brands among our users currently include candy and other snack brands, such as "Dove", "Hershey's", and "Coca-Cola".*

*Of the data quality issues I mentioned, one of the biggest I found was that of duplicate records. Specifically, our products and transactions have rows with duplicate IDs. My guess is that multiple systems -- perhaps those that deal with user records and product records -- are both recording receipt scans into the database, resulting in repeat observations. This is something that will need to be fixed immediately to restore data integrity and also free up storage space. What can I do or to whom I should reach out to investigate this further?*

*I was able to clean the data enough myself to do some simple analysis, and found that candy and other snack brands feature prominently in the top brands among certain users. Focusing on just users who are 21 years or older, the top 5 brands are "Dove", "Nerds", "Coca-Cola", "Great Value", and "Hershey's" in terms of the number of receipts scanned. One potential reason for this particular arrangement of brands is that they could be more likely to be purchased in frequent "small basket" grocery trips. Buying one snack or convenience store item is an easier, and more common way of generating and scanning receipts than getting large baskets of groceries. So it makes sense that snacks would be more likely to be recorded with receipt scans.*

*My recommendation is to investigate this trend further. Once the previously mentioned data integrity issues are dealt with, we should do a further deep dive into this trend by investigating specific products rather than just brands. Unfortunately, our database doesn't record the name of the individual products scanned: only the categories, brands, and manufacturers. If we could bring in or begin collecting this data, it would be extremely valuable to us in the long-run. And, for this investigation in particular, it would allow us to ask 'Among the top brands, which products specifically are being purchased? Are they, generally speaking, bulk consumables or individual snacks?'.*

*I look forward from hearing back from you and continuing this investigation.*

*-Canyen*
___